In [1]:
import os,subprocess
workdir='/mnt/Data1/ribli/expression_code/modelling/'
subprocess.call(['mkdir',workdir])
os.chdir(workdir)

#theano gpu
os.environ['THEANO_FLAGS']='device=gpu'

#import sys
#sys.path.append('../my_modules')
#from loading_utils import read_my_data

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.style.use('ggplot')
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import gc

def read_my_data(fname,**kwargs):
    """Load my data from file into np.arrays.
    
    I had to use garbage collector, because pandas read_csv leaves garbage around.
    """
    
    #load data
    print "Loading data... "
    x=pd.read_csv(fname,sep='\t',header=None)
    
    # for some reason not everything is cleaned up
    #when using the pandas read_csv
    gc.collect()
    
    #probe_id=x[0]
    #y=x.iloc[:,-1].values.astype(np.int8)
    #x=x.iloc[:,1:-1].values.astype(np.int8)
    #return probe_id,x,y
    
    return x[0],x.iloc[:,1:-1].values.astype(np.int8),x.iloc[:,-1].values.astype(np.float64)

In [3]:
train_id,train_x,train_y = read_my_data(fname='../prepare_data/naive_feat_vect.csv')
print train_x.shape

Loading data... 
(234044, 600)


In [6]:
#make it image like
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(np.concatenate([train_x.flatten()]))
train_x=lb.transform(train_x.flatten()).reshape((-1,1,600,5))

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [9]:
train_y=(0.5*(np.sign(train_y-np.median(train_y))+1)).astype(np.int8)

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D,MaxPooling2D

input_dim=train_x.shape[2]
activation='relu'
loss='binary_crossentropy'
optimizer='adadelta'
init='uniform'
pool_size=(8,1)
window_size=5
dense_n=64

model = Sequential()

#Convolution layer 1
model.add(Convolution2D(20,window_size,5, border_mode='valid',input_shape=(1,input_dim,5)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25))

#Convolution layer 2
model.add(Convolution2D(50,window_size,1, border_mode='valid'))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25))

#Dense layer
model.add(Flatten())
model.add(Dense(dense_n,activation=activation))
#model.add(Dropout(0.5))

#final layer
model.add(Dense(1, activation='sigmoid'))

#compile model
model.compile(loss=loss,optimizer=optimizer,class_mode='binary')

In [12]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
import time
def fit_keras_model(model,train_x,train_y,test_x,test_y,validation_split=0.05):
    start=time.time()
    
    #callbacks
    best_model=ModelCheckpoint('best_model',save_best_only=True,verbose=1)
    early_stop=EarlyStopping(patience=7,verbose=1)
    
    #train it
    callb_hist=model.fit(train_x,train_y,nb_epoch = 100 ,verbose=1,
                        validation_split=validation_split,
                         show_accuracy=True,
                        callbacks=[best_model,early_stop])
    #predict
    model.load_weights('best_model')
    train_pred=model.predict_classes(train_x,verbose=1).ravel()
    test_pred=model.predict_classes(test_x,verbose=1).ravel()

    #check errors
    print 'train score:',list((train_pred==train_y)).count(True)/float(len(train_y))
    print 'test score:',list((test_pred==test_y)).count(True)/float(len(test_y))

    print 'It took:',time.time()-start    
    return train_pred,test_pred

In [ ]:
N_train=190000
N_test=40000

train_pred,test_pred=fit_keras_model(
    model,train_x[:N_train],train_y[:N_train],
    train_x[N_train:N_train+N_test],train_y[N_train:N_train+N_test],
    validation_split=0.2)

Train on 152000 samples, validate on 38000 samples
Epoch 1/100
152000/152000 [==============================] - 48s - loss: 0.6790 - acc: 0.5672 - val_loss: 0.6756 - val_acc: 0.5699
Epoch 00000: val_loss improved from inf to 0.67559, saving model to best_model
Epoch 2/100
152000/152000 [==============================] - 48s - loss: 0.6640 - acc: 0.5963 - val_loss: 0.6491 - val_acc: 0.6206
Epoch 00001: val_loss improved from 0.67559 to 0.64909, saving model to best_model
Epoch 3/100
152000/152000 [==============================] - 48s - loss: 0.6558 - acc: 0.6122 - val_loss: 0.6488 - val_acc: 0.6129
Epoch 00002: val_loss improved from 0.64909 to 0.64881, saving model to best_model
Epoch 4/100
152000/152000 [==============================] - 48s - loss: 0.6473 - acc: 0.6233 - val_loss: 0.6312 - val_acc: 0.6430
Epoch 00003: val_loss improved from 0.64881 to 0.63123, saving model to best_model
Epoch 5/100
152000/152000 [==============================] - 48s - loss: 0.6402 - acc: 0.6311 - v